In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from google.colab import files

In [ ]:
# Definisikan kelas Dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
uploaded = files.upload()

Saving labeled_comments.csv to labeled_comments.csv


In [ ]:
# Muat dataset
df = pd.read_csv('labeled_comments.csv')  # Pastikan ada kolom 'text' dan 'label'
texts = df['cleaned_comment'].tolist()
labels = df['label'].tolist()

In [ ]:
# 3. Bagi dataset menjadi train dan validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

In [ ]:
# 4. Muat tokenizer dan model IndoBERT
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1",
    num_labels=2  # Untuk klasifikasi biner
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 5. Buat dataset untuk pelatihan dan validasi
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

In [ ]:
# 6. Definisikan metrik evaluasi
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# 7. Tentukan argumen pelatihan
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',  # Perbaikan: Ganti evaluation_strategy menjadi eval_strategy
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)

In [ ]:
# 8. Inisialisasi Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [ ]:
# 9. Jalankan pelatihan
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fahru-jpg (fahru-jpg-universitas-gunadarma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.212100,0.038400,0.992758,0.991566,0.994118,0.989027
2,0.005700,0.022007,0.995907,0.995243,0.995608,0.994879
3,0.046800,0.028998,0.995592,0.994864,0.997792,0.991953
4,0.000300,0.027163,0.995907,0.995240,0.996334,0.994148
5,0.000100,0.020161,0.997166,0.996709,0.996345,0.997074


TrainOutput(global_step=3970, training_loss=0.03406502510389015, metrics={'train_runtime': 1716.0338, 'train_samples_per_second': 37.007, 'train_steps_per_second': 2.313, 'total_flos': 4177216892659200.0, 'train_loss': 0.03406502510389015, 'epoch': 5.0})

In [ ]:
# 10. Simpan model dan tokenizer
model.save_pretrained('./fine_tuned_indobert')
tokenizer.save_pretrained('./fine_tuned_indobert')

('./fine_tuned_indobert/tokenizer_config.json',
 './fine_tuned_indobert/special_tokens_map.json',
 './fine_tuned_indobert/vocab.txt',
 './fine_tuned_indobert/added_tokens.json',
 './fine_tuned_indobert/tokenizer.json')

In [ ]:
# 11. Contoh prediksi pada data baru
def predict(text):
    # Tentukan perangkat (GPU jika tersedia, kalau tidak CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Pastikan model berada di perangkat yang benar
    model.to(device)

    # Tokenisasi teks
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Pindahkan input ke perangkat yang sama dengan model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Set model ke mode evaluasi
    model.eval()

    # Lakukan prediksi tanpa menghitung gradien
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = outputs.logits.argmax().item()

    return "Judi Online" if prediction == 1 else "Bukan Judi Online"

In [ ]:
# Contoh penggunaan prediksi
sample_text = "ijazah jokowi itu asli penelaah ilmiah itu hanya menebak dan mengira2 dan kemudian di buat fitnah roy suryo adl mentri yang di pecat oleh jokowi sepintas saja tahu kemana larinya mereka"
print(predict(sample_text))

Bukan Judi Online


In [ ]:
# 11. Contoh prediksi pada data baru untuk multiple comments
def predict_multiple(texts):
    # Tentukan perangkat (GPU jika tersedia, kalau tidak CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Pastikan model berada di perangkat yang benar
    model.to(device)

    # Tokenisasi teks
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128, return_token_type_ids=False)

    # Pindahkan input ke perangkat yang sama dengan model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Set model ke mode evaluasi
    model.eval()

    # Lakukan prediksi tanpa menghitung gradien
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=1).tolist()

    results = ["Judi Online" if pred == 1 else "Bukan Judi Online" for pred in predictions]
    return results

In [ ]:

# Contoh penggunaan prediksi untuk multiple comments
sample_texts = [
    "roy suryo itu kan penjahat yang keluar dari penjara kok membuat ulah lagi",
    "semoga roy masuk penjara sebelum tgl juni biar tidak ikut nonton timnas main",
    "ya alloh ya rob penjarakan orang munafik",
    "sehat selalu semuanya salam dari weton88 mudah jackpot",
    "kerennn contentnya kakak cari situs yang benar itu weton88",
    "weton88 tempat paling uhuy"
]
predicted_results = predict_multiple(sample_texts)

for text, result in zip(sample_texts, predicted_results):
    print(f"Komentar: \"{text}\" -> Prediksi: {result}")


Komentar: "roy suryo itu kan penjahat yang keluar dari penjara kok membuat ulah lagi" -> Prediksi: Bukan Judi Online
Komentar: "semoga roy masuk penjara sebelum tgl juni biar tidak ikut nonton timnas main" -> Prediksi: Bukan Judi Online
Komentar: "ya alloh ya rob penjarakan orang munafik" -> Prediksi: Bukan Judi Online
Komentar: "sehat selalu semuanya salam dari weton88 mudah jackpot" -> Prediksi: Judi Online
Komentar: "kerennn contentnya kakak cari situs yang benar itu weton88" -> Prediksi: Judi Online
Komentar: "weton88 tempat paling uhuy" -> Prediksi: Judi Online


In [ ]:
# prompt: simpan model dan tokenizer kedalam drive

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the path in Google Drive where you want to save the model and tokenizer
save_path = '/content/drive/MyDrive/Capstone/Model/fine_tuned_indobert_saved'

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer to the specified path in Google Drive
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to: {save_path}")



Mounted at /content/drive
Model and tokenizer saved to: /content/drive/MyDrive/Capstone/Model/fine_tuned_indobert_saved


In [ ]:
# prompt: load model dari drive dan gunakan untuk predict

# Definisikan path di Google Drive tempat model disimpan
load_path = '/content/drive/MyDrive/Capstone/Model/fine_tuned_indobert_saved'

# Muat tokenizer dan model dari Google Drive
loaded_tokenizer = AutoTokenizer.from_pretrained(load_path)
loaded_model = AutoModelForSequenceClassification.from_pretrained(load_path)

# Perbarui fungsi predict untuk menggunakan model yang dimuat
def predict_from_drive(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loaded_model.to(device)

    inputs = loaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    loaded_model.eval()
    with torch.no_grad():
        outputs = loaded_model(**inputs)
        prediction = outputs.logits.argmax().item()

    return "Judi Online" if prediction == 1 else "Bukan Judi Online"

# Perbarui fungsi predict_multiple untuk menggunakan model yang dimuat
def predict_multiple_from_drive(texts):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loaded_model.to(device)

    inputs = loaded_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128, return_token_type_ids=False)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    loaded_model.eval()
    with torch.no_grad():
        outputs = loaded_model(**inputs)
        predictions = outputs.logits.argmax(dim=1).tolist()

    results = ["Judi Online" if pred == 1 else "Bukan Judi Online" for pred in predictions]
    return results

# Contoh penggunaan prediksi dengan model yang dimuat dari Drive
sample_text_loaded = "ijazah jokowi itu asli penelaah ilmiah itu hanya menebak dan mengira2 dan kemudian di buat fitnah roy suryo adl mentri yang di pecat oleh jokowi sepintas saja tahu kemana larinya mereka"
print("Prediksi dari model yang dimuat dari Drive (teks tunggal):")
print(predict_from_drive(sample_text_loaded))

sample_texts_loaded = [
    "roy suryo itu kan penjahat yang keluar dari penjara kok membuat ulah lagi",
    "semoga roy masuk penjara sebelum tgl juni biar tidak ikut nonton timnas main",
    "ya alloh ya rob penjarakan orang munafik",
    "sehat selalu semuanya salam dari weton88 mudah jackpot",
    "kerennn contentnya kakak cari situs yang benar itu weton88",
    "weton88 tempat paling uhuy"
]
predicted_results_loaded = predict_multiple_from_drive(sample_texts_loaded)

print("\nPrediksi dari model yang dimuat dari Drive (multiple teks):")
for text, result in zip(sample_texts_loaded, predicted_results_loaded):
    print(f"Komentar: \"{text}\" -> Prediksi: {result}")

Prediksi dari model yang dimuat dari Drive (teks tunggal):
Bukan Judi Online

Prediksi dari model yang dimuat dari Drive (multiple teks):
Komentar: "roy suryo itu kan penjahat yang keluar dari penjara kok membuat ulah lagi" -> Prediksi: Bukan Judi Online
Komentar: "semoga roy masuk penjara sebelum tgl juni biar tidak ikut nonton timnas main" -> Prediksi: Bukan Judi Online
Komentar: "ya alloh ya rob penjarakan orang munafik" -> Prediksi: Bukan Judi Online
Komentar: "sehat selalu semuanya salam dari weton88 mudah jackpot" -> Prediksi: Judi Online
Komentar: "kerennn contentnya kakak cari situs yang benar itu weton88" -> Prediksi: Judi Online
Komentar: "weton88 tempat paling uhuy" -> Prediksi: Judi Online
